In [30]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matsim
import os
import travel_events_anls as travel_anls
from tqdm import tqdm

In [27]:
bike_h3Res10 = gpd.read_file(r'../../../../data/clean/freightEmissions/shp/stats/CB_stats_h3Res10.geojson')
car_h3Res10 = gpd.read_file(r'../../../../data/clean/freightEmissions/shp/stats/Van_stats_h3Res10.geojson')
basic_h3Res10 = gpd.read_file(r'../../../../data/clean/freightEmissions/shp/stats/Basic_stats_h3Res10.geojson')

# Indexing network link with spatial unit

In [26]:
network_root_path = r'../../../../data/intermediate/test/freightEmissions/' 

bike_network = matsim.read_network(network_root_path + 'bikeGemeenteLeuvenWithHbefaType.xml.gz')
car_network = matsim.read_network(network_root_path + 'carGemeenteLeuvenWithHbefaType.xml.gz')
raw_network = matsim.read_network(network_root_path + 'GemeenteLeuvenWithHbefaType.xml.gz')

bike_network_gdf = bike_network.as_geo()
car_network_gdf = car_network.as_geo()
raw_network_gdf = raw_network.as_geo()

bike_network_gdf.crs = 'epsg:31370'
car_network_gdf.crs = 'epsg:31370'
raw_network_gdf.crs = 'epsg:31370'

In [78]:
scenario_kws = ['Basic', 'Van', 'CB']
scenario_networks = {kw: road for kw, road in zip(scenario_kws, [raw_network_gdf, car_network_gdf, bike_network_gdf])}
scenario_units = {kw: road for kw, road in zip(scenario_kws, [basic_h3Res10, car_h3Res10, bike_h3Res10  ])}


In [35]:
def agg_network_length_at_unit(network_gdf: gpd.GeoDataFrame, 
                               unit_gdf: gpd.GeoDataFrame,):
    unit_length = {}
    for idx, row in tqdm(network_gdf.iterrows(), total=len(network_gdf)):
        link_geo = row.geometry
        link_id = row['link_id']
        tem_sjoin_gdf = gpd.sjoin(unit_gdf, 
                                gpd.GeoDataFrame(geometry=[link_geo], crs=unit_gdf.crs), 
                                how='left', 
                                predicate='intersects')
        tem_sjoin_gdf = tem_sjoin_gdf.dropna(subset=['h3_index'])
        unit_index_dict = {h3_idx:0 for h3_idx in unit_gdf['h3_index'].tolist()}

        if len(tem_sjoin_gdf) == 0:
            unit_length[link_id] = unit_index_dict

        else:
            num_inter_units = len(tem_sjoin_gdf)
            for i in range(num_inter_units):
                inter_unit_h3_idx = tem_sjoin_gdf.iloc[i]['h3_index'] 
                inter_unit_length = link_geo.intersection(tem_sjoin_gdf.iloc[i]['geometry']).length
                unit_index_dict[inter_unit_h3_idx] += inter_unit_length
            unit_length[link_id] = unit_index_dict

    unit_length_df = pd.DataFrame.from_dict(unit_length, orient='index')
    # unit_length_df = unit_length_df.reset_index()
    # unit_length_df = unit_length_df.rename(columns={'index': 'link_id'})
    return unit_length_df
        

In [37]:
scenario_unit_length_dfs = {}
length_df_output_root = r'../../../../data/clean/freightEmissions/lengthUnitStats/'
os.makedirs(length_df_output_root, exist_ok=True)

for kw in scenario_kws:
    print(f'Calculating network length at unit level for {kw}')
    scenario_unit_length_df = agg_network_length_at_unit(
        scenario_networks[kw], 
        scenario_units[kw])
    scenario_unit_length_dfs[kw] = scenario_unit_length_df
    scenario_unit_length_dfs[kw].to_csv(os.path.join(length_df_output_root, f'{kw}_length_unit.csv.gz'))
    

Calculating network length at unit level for Baisc


100%|██████████| 103753/103753 [40:42<00:00, 42.48it/s]


Calculating network length at unit level for Van


100%|██████████| 52204/52204 [21:07<00:00, 41.17it/s]


Calculating network length at unit level for CB


100%|██████████| 96597/96597 [38:12<00:00, 42.14it/s]


In [51]:
_ = scenario_unit_length_dfs['CB'].loc['95890227_1']
_[_>0]

8a1fa47a1717fff    18.597662
8a1fa47a1737fff    25.243642
Name: 95890227_1, dtype: float64

In [54]:
for _u, _l in _[_>0].items():
    print(f'{_u}: {_l}')
    print(f'{type(_u)}: {type(_l)}')

8a1fa47a1717fff: 18.597661511562357
<class 'str'>: <class 'float'>
8a1fa47a1737fff: 25.243642457915367
<class 'str'>: <class 'float'>


In [ ]:
scenario_unit_length_dfs['CB'].loc['95890227_1'][['8a1fa47a1717fff','8a1fa47a1737fff']]

8a1fa47a1717fff    18.597662
8a1fa47a1737fff    25.243642
Name: 95890227_1, dtype: float64

# Agg vkt with spatial unit

In [81]:
scenario_link_length_units={}
scenario_kws = ['Basic', 'Van', 'CB']
length_unit_root = r'../../../../data/clean/freightEmissions/lengthUnitStats/'
for kw in scenario_kws:
    scenario_link_length_units[kw] = pd.read_csv(os.path.join(length_unit_root, f'{kw}_length_unit.csv.gz'), index_col=0)
    scenario_link_length_units[kw].index = scenario_link_length_units[kw].index.astype(str)
    scenario_link_length_units[kw] = scenario_link_length_units[kw].astype(float)

In [60]:
events_root_prefix = r'../../../../data/intermediate/test/freightEmissions/'
iter_list = list(range(300, 400))

In [58]:
def agg_vkt_at_unit(events_dir: str,
                    link_length_units_df: pd.DataFrame, 
                    unit_gdf: gpd.GeoDataFrame,):
    travel_events = travel_anls.read_travel_events(events_dir)
    v_travel_chains, v_travel_times = travel_anls.derive_vehicle_travel_chain(travel_events_df=travel_events)
    
    unit_dict = {h3_idx:0 for h3_idx in unit_gdf['h3_index'].tolist()}

    for v, chain in v_travel_chains.items():
        for link_id in chain:
            length_units_row = link_length_units_df.loc[link_id]
            for unit_, length_ in length_units_row[length_units_row>0].items():
                unit_dict[unit_] += length_
                
    return unit_dict

In [96]:
scenario_vkt_unit_dicts = {scenario_kw:{iter_:None for iter_ in iter_list} for scenario_kw in scenario_kws}
total_operations = len(scenario_kws) * len(iter_list)
progress_bar = tqdm(total=total_operations, desc='Calculating VKT at unit level')
for kw in scenario_kws:
    print(f'Calculating VKT at unit level for {kw}')
    for iter_ in iter_list:
        iter_events_dir = os.path.join(events_root_prefix, f'scenario{kw}', f'iter{iter_}', f'outputs/') 
        
        iter_unit_vkt_dict = agg_vkt_at_unit(
            events_dir=iter_events_dir, 
            link_length_units_df=scenario_link_length_units[kw], 
            unit_gdf=scenario_units[kw]
            )
        scenario_vkt_unit_dicts[kw][iter_] = iter_unit_vkt_dict 
        progress_bar.update(1)

progress_bar.close()
    

Calculating VKT at unit level:   0%|          | 0/300 [00:00<?, ?it/s]

Calculating VKT at unit level for Basic


Calculating VKT at unit level:  33%|███▎      | 100/300 [00:23<00:46,  4.31it/s]

Calculating VKT at unit level for Van


Calculating VKT at unit level:  67%|██████▋   | 200/300 [00:52<00:28,  3.54it/s]

Calculating VKT at unit level for CB


Calculating VKT at unit level: 100%|██████████| 300/300 [02:08<00:00,  2.34it/s]


In [109]:
scenario_vkt_unit_dfs = {}
vkt_geofile_output_root = r'../../../../data/clean/freightEmissions/shp/statsWithVKT/'
os.makedirs(vkt_geofile_output_root, exist_ok=True)
for kw in scenario_kws:
    _df = pd.DataFrame(scenario_vkt_unit_dicts[kw])
    _df['mean_vkt'] = _df.mean(axis=1) / 1000  # km

    _df['median_vkt'] = _df.median(axis=1) / 1000
    scenario_vkt_unit_dfs[kw] = _df
    scenario_vkt_unit_dfs[kw].to_csv(os.path.join(length_df_output_root, f'{kw}_vkt_unit.csv.gz'))

    _df = _df.reset_index()
    _df = _df.rename(columns={'index': 'h3_index'})
    # Merge with unit gdf
    scen_unit_gdf = scenario_units[kw]
    scen_unit_gdf = scen_unit_gdf.merge(_df[['h3_index','mean_vkt', 'median_vkt']], on='h3_index',  how='left')

    # Output to geojson
    scen_unit_gdf.to_file(os.path.join(vkt_geofile_output_root, f'{kw}_vkt_unit.geojson'), driver='GeoJSON')
    

In [106]:
scenario_vkt_unit_dfs['Basic']

,300,301,302,303,304,305,306,307,308,309,...,392,393,394,395,396,397,398,399,mean_vkt,median_vkt
8a1fa47ae9b7fff,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,162.632600,0.000000,148.258843,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029567,0.000000
8a1fa47a0257fff,234.696646,234.696646,117.348323,234.696646,117.348323,234.696646,117.348323,117.348323,117.348323,234.696646,...,117.348323,117.348323,117.348323,117.348323,117.348323,117.348323,117.348323,117.348323,0.125563,0.117348
8a1fa47a1887fff,159.124009,0.000000,145.868039,205.267051,249.023962,112.252154,236.499263,20.579361,0.000000,144.058400,...,220.749128,0.000000,403.551272,145.057971,0.000000,27.874633,20.579361,0.000000,0.099739,0.114749
8a1fa47a02effff,334.314219,281.860052,345.865406,323.335789,208.892330,323.335789,161.667895,120.192157,208.892330,334.314219,...,243.746857,161.667895,245.241626,172.646324,161.667895,216.343289,217.477045,222.799449,0.196517,0.192996
8a1fa47124f7fff,209.044443,94.618095,201.295747,223.533433,0.000000,201.295747,0.000000,114.426349,0.000000,0.000000,...,337.959782,355.391011,0.000000,0.000000,223.533433,94.618095,116.855780,127.050140,0.106882,0.106678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a1fa47a0acffff,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001176,0.000000
8a1fa47a0347fff,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001468,0.000000
8a1fa47a0377fff,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000174,0.000000
8a1fa47a035ffff,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001179,0.000000


In [107]:
scenario_vkt_unit_dfs['Basic'].dropna()

,300,301,302,303,304,305,306,307,308,309,...,392,393,394,395,396,397,398,399,mean_vkt,median_vkt
8a1fa47ae9b7fff,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,162.632600,0.000000,148.258843,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029567,0.000000
8a1fa47a0257fff,234.696646,234.696646,117.348323,234.696646,117.348323,234.696646,117.348323,117.348323,117.348323,234.696646,...,117.348323,117.348323,117.348323,117.348323,117.348323,117.348323,117.348323,117.348323,0.125563,0.117348
8a1fa47a1887fff,159.124009,0.000000,145.868039,205.267051,249.023962,112.252154,236.499263,20.579361,0.000000,144.058400,...,220.749128,0.000000,403.551272,145.057971,0.000000,27.874633,20.579361,0.000000,0.099739,0.114749
8a1fa47a02effff,334.314219,281.860052,345.865406,323.335789,208.892330,323.335789,161.667895,120.192157,208.892330,334.314219,...,243.746857,161.667895,245.241626,172.646324,161.667895,216.343289,217.477045,222.799449,0.196517,0.192996
8a1fa47124f7fff,209.044443,94.618095,201.295747,223.533433,0.000000,201.295747,0.000000,114.426349,0.000000,0.000000,...,337.959782,355.391011,0.000000,0.000000,223.533433,94.618095,116.855780,127.050140,0.106882,0.106678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a1fa47a0acffff,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001176,0.000000
8a1fa47a0347fff,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001468,0.000000
8a1fa47a0377fff,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000174,0.000000
8a1fa47a035ffff,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001179,0.000000


# Test on identification of travel trips

In [2]:
test_events_dir = r'../../../../data/intermediate/test/freightEmissions/scenarioBasic/iter300/outputs/'


In [77]:
test_events_dir[62:]

'sic/iter300/outputs/'

In [75]:
for _, __ in enumerate(test_events_dir):
    if __ != iter_events_dir[_]:
        print('Not the same')
        print(f'{_}: {__}')
    else:
        pass

Not the same
62: s
Not the same
63: i
Not the same
73: /
Not the same
74: o
Not the same
75: u
Not the same
76: t
Not the same
77: p
Not the same
78: u
Not the same
79: t
Not the same
80: s


IndexError: string index out of range

In [70]:
test_travel_events = travel_anls.read_travel_events(test_events_dir)
test_travel_events

,time,vehicle,type,link
0,108.0,freight_carrier_0_veh_carrier0_Van_0_1,left link,25806807_r_1
1,108.0,freight_carrier_0_veh_carrier0_Van_0_1,entered link,25806807_r_2
2,110.0,freight_carrier_0_veh_carrier0_Van_0_1,left link,25806807_r_2
3,110.0,freight_carrier_0_veh_carrier0_Van_0_1,entered link,26409820_2
4,111.0,freight_carrier_0_veh_carrier0_Van_0_1,left link,26409820_2
...,...,...,...,...
4225,24272.0,freight_carrier_0_veh_carrier0_Van_0_1,entered link,10155462_1-10155462_2-10155462_3
4226,24273.0,freight_carrier_0_veh_carrier0_Van_0_1,left link,10155462_1-10155462_2-10155462_3
4227,24273.0,freight_carrier_0_veh_carrier0_Van_0_1,entered link,25806807_r_0
4228,24275.0,freight_carrier_0_veh_carrier0_Van_0_1,left link,25806807_r_0


In [5]:
test_v_travel_chains, test_v_travel_times = travel_anls.derive_vehicle_travel_chain(test_travel_events 
                                                             )
test_v_travel_chains

{'freight_carrier_0_veh_carrier0_Van_0_1': ['25806807_r_1',
  '25806807_r_2',
  '26409820_2',
  '26409820_3',
  '26409820_4',
  '26409820_5',
  '26409820_6',
  '26409819_0',
  '25933426_r_7',
  '25933426_r_8',
  '25933426_r_9',
  '25933426_r_10',
  '25933426_r_11',
  '3876851_8',
  '3876851_9',
  '3876851_10',
  '3876851_11',
  '3876851_12',
  '3876851_13',
  '3876851_14',
  '3876851_15-3876851_16',
  '3876851_17',
  '3876851_18',
  '3876851_19-3876851_20',
  '337406053_35',
  '337406053_36',
  '337406053_37',
  '337406053_38',
  '337406053_39',
  '337406053_40',
  '337758975_0',
  '3869819_0',
  '3869819_1',
  '3869819_2',
  '337406021_r_0-337406021_r_1-337406021_r_2',
  '608715621_r_0',
  '377918611_r_0',
  '377918611_1',
  '608715621_0',
  '425777009_0-425777009_1-425777009_2',
  '425777009_3-425777009_4',
  '425777010_0-701316144_0',
  '701316144_1',
  '701316144_2',
  '29283599_0',
  '29283599_1',
  '29283599_2',
  '29283599_3-29283599_4',
  '124190277_0-124190277_1',
  '124190277

In [48]:
basic_unit_dict = {h3_idx:0 for h3_idx in basic_h3Res10['h3_index'].tolist()}
basic_unit_dict

{'8a1fa47ae9b7fff': 0,
 '8a1fa47a0257fff': 0,
 '8a1fa47a1887fff': 0,
 '8a1fa47a02effff': 0,
 '8a1fa47124f7fff': 0,
 '8a1fa47a1ae7fff': 0,
 '8a1fa47a1757fff': 0,
 '8a1fa47a1c17fff': 0,
 '8a1fa47a1caffff': 0,
 '8a1fa47a1197fff': 0,
 '8a1fa47a1627fff': 0,
 '8a1fa47a158ffff': 0,
 '8a1fa47a1527fff': 0,
 '8a1fa47a145ffff': 0,
 '8a1fa47a1657fff': 0,
 '8a1fa47a3967fff': 0,
 '8a1fa47a16effff': 0,
 '8a1fa47a1787fff': 0,
 '8a1fa47a528ffff': 0,
 '8a1fa47a18b7fff': 0,
 '8a1fa47a194ffff': 0,
 '8a1fa471248ffff': 0,
 '8a1fa47a1297fff': 0,
 '8a1fa47a1daffff': 0,
 '8a1fa47a1b17fff': 0,
 '8a1fa47a1baffff': 0,
 '8a1fa47a1d17fff': 0,
 '8a1fa47a56effff': 0,
 '8a1fa47a1097fff': 0,
 '8a1fa47a561ffff': 0,
 '8a1fa47a1c47fff': 0,
 '8a1fa47a1cdffff': 0,
 '8a1fa47a5657fff': 0,
 '8a1fa47a10cffff': 0,
 '8a1fa47a1d77fff': 0,
 '8a1fa47a138ffff': 0,
 '8a1fa47a1427fff': 0,
 '8a1fa47a1557fff': 0,
 '8a1fa47a1ab7fff': 0,
 '8a1fa47a171ffff': 0,
 '8a1fa47a1a1ffff': 0,
 '8a1fa47a17b7fff': 0,
 '8a1fa47a184ffff': 0,
 '8a1fa47a1

In [55]:
total_operations = sum(len(chain) for chain in test_v_travel_chains.values())
progress_bar = tqdm(total=total_operations, desc="Processing links")

for v, chain in test_v_travel_chains.items():
    for link_ in chain:
        _length_units_row = scenario_unit_length_dfs['Baisc'].loc[link_]
        for unit_, length_ in _length_units_row[_length_units_row>0].items():
            basic_unit_dict[unit_] += length_
        progress_bar.update(1)

progress_bar.close()
basic_unit_dict_df = pd.DataFrame.from_dict(basic_unit_dict, orient='index')


Processing links: 100%|██████████| 2116/2116 [00:00<00:00, 9524.53it/s]


In [56]:
basic_unit_dict_df

,0
8a1fa47ae9b7fff,0.000000
8a1fa47a0257fff,234.696646
8a1fa47a1887fff,159.124009
8a1fa47a02effff,334.314219
8a1fa47124f7fff,209.044443
...,...
8a1fa47a0acffff,0.000000
8a1fa47a0347fff,0.000000
8a1fa47a0377fff,0.000000
8a1fa47a035ffff,0.000000
